In [ ]:
!unzip clean_partial.zip

Archive:  clean_partial.zip
   creating: clean_partial/
   creating: clean_partial/Eurasian Eagle Owl (Bubo Bubo)/
  inflating: clean_partial/Eurasian Eagle Owl (Bubo Bubo)/XC521226 - Eurasian Eagle-Owl - Bubo bubo(1)_1.wav  
  inflating: clean_partial/Eurasian Eagle Owl (Bubo Bubo)/XC521226 - Eurasian Eagle-Owl - Bubo bubo(1)_14.wav  
  inflating: clean_partial/Eurasian Eagle Owl (Bubo Bubo)/XC521226 - Eurasian Eagle-Owl - Bubo bubo(1)_15.wav  
  inflating: clean_partial/Eurasian Eagle Owl (Bubo Bubo)/XC521226 - Eurasian Eagle-Owl - Bubo bubo(1)_2.wav  
  inflating: clean_partial/Eurasian Eagle Owl (Bubo Bubo)/XC521226 - Eurasian Eagle-Owl - Bubo bubo(1)_8.wav  
  inflating: clean_partial/Eurasian Eagle Owl (Bubo Bubo)/XC521226 - Eurasian Eagle-Owl - Bubo bubo(1)_9.wav  
  inflating: clean_partial/Eurasian Eagle Owl (Bubo Bubo)/XC521226 - Eurasian Eagle-Owl - Bubo bubo_14.wav  
  inflating: clean_partial/Eurasian Eagle Owl (Bubo Bubo)/XC521226 - Eurasian Eagle-Owl - Bubo bubo_2.wav  


In [ ]:
!unzip Audio-Classification\ -\ Copy.zip

Archive:  Audio-Classification - Copy.zip
   creating: models/
  inflating: models/.gitignore       
   creating: notebooks/
  inflating: notebooks/Confusion Matrix and ROC.ipynb  
  inflating: notebooks/Google Slides Plots.ipynb  
  inflating: notebooks/Kapre Mel Spectrogram.ipynb  
  inflating: notebooks/Plot History.ipynb  
  inflating: notebooks/Thumbnail plots.ipynb  
   creating: docs/
  inflating: docs/conf_mat.png       
  inflating: docs/kapre.pdf          
  inflating: docs/mel_spectrograms.png  
  inflating: docs/model_history.png  
  inflating: docs/roc.png            
  inflating: docs/signal_envelope.png  
  inflating: docs/time_series.png    
   creating: logs/
  inflating: logs/conv1d_history.csv  
  inflating: logs/conv2d_history.csv  
  inflating: logs/lstm_history.csv   
  inflating: logs/y_pred.npy         


In [ ]:
!unzip Oriental_Scops_Owl.zip

Archive:  Oriental_Scops_Owl.zip
   creating: Oriental Scops Owl (Otus sunia)/
  inflating: Oriental Scops Owl (Otus sunia)/XC19792 - Oriental Scops Owl - Otus sunia.wav  
  inflating: Oriental Scops Owl (Otus sunia)/XC214019 - Oriental Scops Owl - Otus sunia(1).wav  
  inflating: Oriental Scops Owl (Otus sunia)/XC214019 - Oriental Scops Owl - Otus sunia.wav  
  inflating: Oriental Scops Owl (Otus sunia)/XC217789-1047-OrientalScopsOwl-2(1).wav  
  inflating: Oriental Scops Owl (Otus sunia)/XC217789-1047-OrientalScopsOwl-2(2).wav  
  inflating: Oriental Scops Owl (Otus sunia)/XC217789-1047-OrientalScopsOwl-2(3).wav  
  inflating: Oriental Scops Owl (Otus sunia)/XC217789-1047-OrientalScopsOwl-2.wav  
  inflating: Oriental Scops Owl (Otus sunia)/XC217789-1047-OrientalScopsOwl.wav  
  inflating: Oriental Scops Owl (Otus sunia)/XC267356 - Oriental Scops Owl - Otus sunia.wav  
  inflating: Oriental Scops Owl (Otus sunia)/XC312463 - Oriental Scops Owl - Otus sunia.wav  
  inflating: Oriental 

In [ ]:
!unzip Mottled_Wood_Owl.zip

Archive:  Mottled_Wood_Owl.zip
   creating: Mottled Wood Owl (Strix ocellata)/
  inflating: Mottled Wood Owl (Strix ocellata)/XC128102 - Mottled Wood Owl - Strix ocellata grisescens(1).wav  
  inflating: Mottled Wood Owl (Strix ocellata)/XC128102 - Mottled Wood Owl - Strix ocellata grisescens(2).wav  
  inflating: Mottled Wood Owl (Strix ocellata)/XC128102 - Mottled Wood Owl - Strix ocellata grisescens(3).wav  
  inflating: Mottled Wood Owl (Strix ocellata)/XC128102 - Mottled Wood Owl - Strix ocellata grisescens(4).wav  
  inflating: Mottled Wood Owl (Strix ocellata)/XC128102 - Mottled Wood Owl - Strix ocellata grisescens(5).wav  
  inflating: Mottled Wood Owl (Strix ocellata)/XC128102 - Mottled Wood Owl - Strix ocellata grisescens(6).wav  
  inflating: Mottled Wood Owl (Strix ocellata)/XC149334 - Mottled Wood Owl - Strix ocellata(1).wav  
  inflating: Mottled Wood Owl (Strix ocellata)/XC149334 - Mottled Wood Owl - Strix ocellata(2).wav  
  inflating: Mottled Wood Owl (Strix ocellata)/

In [ ]:
from keras import layers
from keras.layers import TimeDistributed
from keras.models import Model
from keras.regularizers import l2
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
import os

from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.utils import to_categorical
import os
from scipy.io import wavfile
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from glob import glob
import argparse



def Conv1D(N_CLASSES=10, SR=16000, DT=1.0):
    i = layers.Input(shape=(1, int(SR*DT)), name='input')
    x = Melspectrogram(n_dft=512, n_hop=160,
                       padding='same', sr=SR, n_mels=128,
                       fmin=0.0, fmax=SR/2, power_melgram=2.0,
                       return_decibel_melgram=True, trainable_fb=False,
                       trainable_kernel=False,
                       name='melbands')(i)
    x = Normalization2D(str_axis='batch', name='batch_norm')(x)
    x = layers.Permute((2,1,3), name='permute')(x)
    x = TimeDistributed(layers.Conv1D(8, kernel_size=(4), activation='tanh'), name='td_conv_1d_tanh')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), name='max_pool_2d_1')(x)
    x = TimeDistributed(layers.Conv1D(16, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_1')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), name='max_pool_2d_2')(x)
    x = TimeDistributed(layers.Conv1D(32, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_2')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), name='max_pool_2d_3')(x)
    x = TimeDistributed(layers.Conv1D(64, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_3')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), name='max_pool_2d_4')(x)
    x = TimeDistributed(layers.Conv1D(128, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_4')(x)
    x = layers.GlobalMaxPooling2D(name='global_max_pooling_2d')(x)
    x = layers.Dropout(rate=0.1, name='dropout')(x)
    x = layers.Dense(64, activation='relu', activity_regularizer=l2(0.001), name='dense')(x)
    o = layers.Dense(N_CLASSES, activation='softmax', name='softmax')(x)

    model = Model(inputs=i, outputs=o, name='1d_convolution')
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


def Conv2D(N_CLASSES=10, SR=16000, DT=1.0):
    i = layers.Input(shape=(1, int(SR*DT)), name='input')
    x = Melspectrogram(n_dft=512, n_hop=160,
                       padding='same', sr=SR, n_mels=128,
                       fmin=0.0, fmax=SR/2, power_melgram=2.0,
                       return_decibel_melgram=True, trainable_fb=False,
                       trainable_kernel=False,
                       name='melbands')(i)
    x = Normalization2D(str_axis='batch', name='batch_norm')(x)
    x = layers.Conv2D(8, kernel_size=(7,7), activation='tanh', padding='same', name='conv2d_tanh')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), padding='same', name='max_pool_2d_1')(x)
    x = layers.Conv2D(16, kernel_size=(5,5), activation='relu', padding='same', name='conv2d_relu_1')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), padding='same', name='max_pool_2d_2')(x)
    x = layers.Conv2D(16, kernel_size=(3,3), activation='relu', padding='same', name='conv2d_relu_2')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), padding='same', name='max_pool_2d_3')(x)
    x = layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same', name='conv2d_relu_3')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), padding='same', name='max_pool_2d_4')(x)
    x = layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same', name='conv2d_relu_4')(x)
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dropout(rate=0.2, name='dropout')(x)
    x = layers.Dense(64, activation='relu', activity_regularizer=l2(0.001), name='dense')(x)
    o = layers.Dense(N_CLASSES, activation='softmax', name='softmax')(x)

    model = Model(inputs=i, outputs=o, name='2d_convolution')
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


def LSTM(N_CLASSES=10, SR=16000, DT=1.0):
    i = layers.Input(shape=(1, int(SR*DT)), name='input')
    x = Melspectrogram(n_dft=512, n_hop=160,
                       padding='same', sr=SR, n_mels=128,
                       fmin=0.0, fmax=SR/2, power_melgram=2.0,
                       return_decibel_melgram=True, trainable_fb=False,
                       trainable_kernel=False,
                       name='melbands')(i)
    x = Normalization2D(str_axis='batch', name='batch_norm')(x)
    x = layers.Permute((2,1,3), name='permute')(x)
    x = TimeDistributed(layers.Reshape((-1,)), name='reshape')(x)
    s = TimeDistributed(layers.Dense(64, activation='tanh'),
                        name='td_dense_tanh')(x)
    x = layers.Bidirectional(layers.LSTM(32, return_sequences=True),
                             name='bidirectional_lstm')(s)
    x = layers.concatenate([s, x], axis=2, name='skip_connection')
    x = layers.Dense(64, activation='relu', name='dense_1_relu')(x)
    x = layers.MaxPooling1D(name='max_pool_1d')(x)
    x = layers.Dense(32, activation='relu', name='dense_2_relu')(x)
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dropout(rate=0.2, name='dropout')(x)
    x = layers.Dense(32, activation='relu',
                         activity_regularizer=l2(0.001),
                         name='dense_3_relu')(x)
    o = layers.Dense(N_CLASSES, activation='softmax', name='softmax')(x)

    model = Model(inputs=i, outputs=o, name='long_short_term_memory')
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model



class DataGenerator(keras.utils.Sequence):
    def __init__(self, wav_paths, labels, sr, dt, n_classes,
                 batch_size=32, shuffle=True):
        self.wav_paths = wav_paths
        self.labels = labels
        self.sr = sr
        self.dt = dt
        self.n_classes = n_classes
        self.batch_size = batch_size
        self.shuffle = True
        self.on_epoch_end()


    def __len__(self):
        return int(np.floor(len(self.wav_paths) / self.batch_size))


    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        wav_paths = [self.wav_paths[k] for k in indexes]
        labels = [self.labels[k] for k in indexes]

        # generate a batch of time data
        X = np.empty((self.batch_size, 1, int(self.sr*self.dt)), dtype=np.int16)
        Y = np.empty((self.batch_size, self.n_classes), dtype=np.float32)

        for i, (path, label) in enumerate(zip(wav_paths, labels)):
            rate, wav = wavfile.read(path)
            X[i,] = wav.reshape(1, -1)
            Y[i,] = to_categorical(label, num_classes=self.n_classes)

        return X, Y


    def on_epoch_end(self):
        self.indexes = np.arange(len(self.wav_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)


def train(args):
    src_root = args.src_root
    sr = args.sample_rate
    dt = args.delta_time
    batch_size = args.batch_size
    model_type = args.model_type
    params = {'N_CLASSES':len(os.listdir(args.src_root)),
              'SR':sr,
              'DT':dt}
    models = {'conv1d':Conv1D(**params),
              'conv2d':Conv2D(**params),
              'lstm':  LSTM(**params)}
    assert model_type in models.keys(), '{} not an available model'.format(model_type)
    csv_path = os.path.join('logs', '{}_history.csv'.format(model_type))

    wav_paths = glob('{}/**'.format(src_root), recursive=True)
    wav_paths = [x.replace(os.sep, '/') for x in wav_paths if '.wav' in x]
    classes = sorted(os.listdir(args.src_root))
    le = LabelEncoder()
    le.fit(classes)
    labels = [os.path.split(x)[0].split('/')[-1] for x in wav_paths]
    labels = le.transform(labels)

    wav_train, wav_val, label_train, label_val = train_test_split(wav_paths,
                                                                  labels,
                                                                  test_size=0.1,
                                                                  random_state=0)

    assert len(label_train) >= args.batch_size, 'number of train samples must be >= batch_size'

    tg = DataGenerator(wav_train, label_train, sr, dt,
                       len(set(label_train)), batch_size=batch_size)
    vg = DataGenerator(wav_val, label_val, sr, dt,
                       len(set(label_val)), batch_size=batch_size)

    model = models[model_type]
    cp = ModelCheckpoint('models/{}.h5'.format(model_type), monitor='val_loss',
                         save_best_only=True, save_weights_only=False,
                         mode='auto', verbose=1)
    csv_logger = CSVLogger(csv_path, append=False)
    model.fit(tg, validation_data=vg,
              epochs=30, verbose=1,
              callbacks=[csv_logger, cp])


if __name__ == '__main__':

    parser = argparse.ArgumentParser(description='Audio Classification Training')
    parser.add_argument('--model_type', type=str, default='lstm',
                        help='model to run. i.e. conv1d, conv2d, lstm')
    parser.add_argument('--src_root', type=str, default='clean_partial',
                        help='directory of audio files in total duration')
    parser.add_argument('--batch_size', type=int, default=16,
                        help='batch size')
    parser.add_argument('--delta_time', '-dt', type=float, default=1.0,
                        help='time in seconds to sample audio')
    parser.add_argument('--sample_rate', '-sr', type=int, default=16000,
                        help='sample rate of clean audio')
    args, _ = parser.parse_known_args()

    train(args)


tracking <tf.Variable 'melbands_14/real_kernels:0' shape=(512, 1, 1, 257) dtype=float32> dft_real_kernels
tracking <tf.Variable 'melbands_14/imag_kernels:0' shape=(512, 1, 1, 257) dtype=float32> dft_imag_kernels
tracking <tf.Variable 'melbands_14/Variable:0' shape=(257, 128) dtype=float32> freq2mel
tracking <tf.Variable 'melbands_15/real_kernels:0' shape=(512, 1, 1, 257) dtype=float32> dft_real_kernels
tracking <tf.Variable 'melbands_15/imag_kernels:0' shape=(512, 1, 1, 257) dtype=float32> dft_imag_kernels
tracking <tf.Variable 'melbands_15/Variable:0' shape=(257, 128) dtype=float32> freq2mel
tracking <tf.Variable 'melbands_16/real_kernels:0' shape=(512, 1, 1, 257) dtype=float32> dft_real_kernels
tracking <tf.Variable 'melbands_16/imag_kernels:0' shape=(512, 1, 1, 257) dtype=float32> dft_imag_kernels
tracking <tf.Variable 'melbands_16/Variable:0' shape=(257, 128) dtype=float32> freq2mel
Epoch 1/30
63/63 [==============================] - 39s 613ms/step - loss: 0.8495 - accuracy: 0.7917

In [ ]:
from keras.models import load_model
from librosa.core import resample, to_mono


def envelope(y, rate, threshold):
    mask = []
    y = pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/20),
                       min_periods=1,
                       center=True).max()
    for mean in y_mean:
        if mean > threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask, y_mean


def downsample_mono(path, sr):
    rate, wav = wavfile.read(path)
    wav = wav.astype(np.float32, order='F')
    try:
        tmp = wav.shape[1]
        wav = to_mono(wav.T)
    except:
        pass
    wav = resample(wav, rate, sr)
    wav = wav.astype(np.int16)
    return sr, wav

def make_prediction(args):

    model = load_model(args.model_fn,
        custom_objects={'Melspectrogram':Melspectrogram,
                        'Normalization2D':Normalization2D})

    wav_paths = glob('{}/**'.format(args.src_dir), recursive=True)
    wav_paths = sorted([x.replace(os.sep, '/') for x in wav_paths if '.wav' in x])
    classes = sorted(os.listdir(args.src_dir))
    labels = [os.path.split(x)[0].split('/')[-1] for x in wav_paths]
    le = LabelEncoder()
    y_true = le.fit_transform(labels)
    results = []

    for z, wav_fn in tqdm(enumerate(wav_paths), total=len(wav_paths)):
        rate, wav = downsample_mono(wav_fn, args.sr)
        mask, env = envelope(wav, rate, threshold=args.threshold)
        clean_wav = wav[mask]
        step = int(args.sr*args.dt)
        batch = []

        for i in range(0, clean_wav.shape[0], step):
            sample = clean_wav[i:i+step]
            sample = sample.reshape(1,-1)
            if sample.shape[0] < step:
                tmp = np.zeros(shape=(1,step), dtype=np.int16)
                tmp[:,:sample.shape[1]] = sample.flatten()
                sample = tmp
            batch.append(sample)
        X_batch = np.array(batch)
        y_pred = model.predict(X_batch)
        y_mean = np.mean(y_pred, axis=0)
        y_pred = np.argmax(y_mean)
        real_class = os.path.dirname(wav_fn).split('/')[-1]
        print('Actual class: {}, Predicted class: {}'.format(real_class, classes[y_pred]))
        results.append(y_mean)

    np.save(os.path.join('logs', args.pred_fn), np.array(results))


if __name__ == '__main__':

    parser = argparse.ArgumentParser(description='Audio Classification Training')
    parser.add_argument('--model_fn', type=str, default='models/lstm.h5',
                        help='model file to make predictions')
    parser.add_argument('--pred_fn', type=str, default='y_pred',
                        help='fn to write predictions in logs dir')
    parser.add_argument('--src_dir', type=str, default='test',
                        help='directory containing wavfiles to predict')
    parser.add_argument('--dt', type=float, default=1.0,
                        help='time in seconds to sample audio')
    parser.add_argument('--sr', type=int, default=16000,
                        help='sample rate of clean audio')
    parser.add_argument('--threshold', type=str, default=20,
                        help='threshold magnitude for np.int16 dtype')
    args, _ = parser.parse_known_args()

    make_prediction(args)


int_axis=-1 passed but is ignored, str_axis is used instead.
tracking <tf.Variable 'melbands_21/real_kernels:0' shape=(512, 1, 1, 257) dtype=float32> dft_real_kernels
tracking <tf.Variable 'melbands_21/imag_kernels:0' shape=(512, 1, 1, 257) dtype=float32> dft_imag_kernels
tracking <tf.Variable 'melbands_21/Variable:0' shape=(257, 128) dtype=float32> freq2mel


  0%|          | 0/76 [00:00<?, ?it/s]


IndexError: ignored